## HW 1: Sinusoids, Functions, Additive Synthesis (15 points)
For each markdown cell, add a cell (or cells) of code below. 
Reminders:
* This is an individual assignment. 
* If you use GenAI tools to assist you with your homework, remember to fill out the GenAI Usage Statement at the bottom of the notebook. Even if you use GenAI, you should not be directly copying the code.
* You may only use functions/packages we have discussed in class

In [4]:
from IPython.display import Math, Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib


### Question 1 - Sinusoids (3 points)
Define the function, **genSine(f, ...)** that construct a sinusoid at frequency `f`.
The arguments should include:  
* amplitude, with default value of 1 (float)
* sample rate, with default value of 44.1kHz (float)
* time in seconds, with default value of 1 (float)
* phase offset, with default value of 0 (float)
    
and will return the numpy array containing the sinusoidal waveform.

Your function must check for appropriate input and handle any errors.
    

In [5]:
def genSine(f, a=1, fs=44100, t=1, phi=0):

    n_samples = int(fs * t)
    time = np.linspace(0, t, n_samples, endpoint=False)

    sine = a* np.sin(2 * np.pi * f * time + phi)

    return sine


### Question 2 - Classical Waveform (5 points)
Define a function, **genWave(freq, t, numHarms=None, A=1, phi=0, fs=44100)**, that will create one of the fundamental waveforms (saw, triangle, or square) built from the combination of sinusoids at integer multiples of a fundamental frequency.  The arguments passed will be as follows:

* freq will be the frequency in Hz (int or float).
* t will be the length of time in seconds (int or float).
* numHarms will be the number of harmonics used. If no value is given (NoneType), you should create the maximum harmonics possible given the sampling rate

The function should return a numpy array. **You may not use existing waveform functions such as those in the `scipy` library.**

Your function must check for appropriate input and handle any errors.

Try to make your function as efficient as you can, but we will not take points off for a long run time. Be mindful by using NumPy functions rather than direct iteration.

*Hint: your function should use your genSine function

**Bonus: Expand this function to create any of the classical waveforms with a new argument "type" specifying which to build.

In [6]:
def genWave(freq, t, numHarms=None, A=1.0, phi=0.0, fs=44100.0, type="saw"):
    for name, val in {
        "frequency": freq,
        "time": t,
        "amplitude": A,
        "sample rate": fs
    }.items():
        if not isinstance(val, (int, float)):
            raise TypeError(f"{name} must be numeric")

    if freq <= 0 or fs <= 0 or t <= 0:
        raise ValueError("frequency, time, and sample rate must be > 0")

    if type not in {"saw", "square", "triangle"}:
        raise ValueError("type must be 'saw', 'square', or 'triangle'")

    nyquist = fs / 2
    maxHarms = int(nyquist // freq)

    if numHarms is None:
        numHarms = maxHarms
    else:
        if not isinstance(numHarms, int) or numHarms <= 0:
            raise ValueError("numHarms must be a positive integer or None")
        numHarms = min(numHarms, maxHarms)

    wave = np.zeros(int(fs * t))

    if type == "saw":
        
        for n in range(1, numHarms + 1):
            wave += genSine(n * freq, a=1/n, fs=fs, t=t, phi=phi)

    elif type == "square":
        
        for n in range(1, 2 * numHarms, 2):
            wave += genSine(n * freq, a=1/n, fs=fs, t=t, phi=phi)

    elif type == "triangle":
        
        sign = 1
        for n in range(1, 2 * numHarms, 2):
            wave += sign * genSine(n * freq, a=1/(n**2), fs=fs, t=t, phi=phi)
            sign *= -1

    wave /= np.max(np.abs(wave))
    wave *= A

    return wave




### Question 3 - Arpeggio (4 points)

Define the function arpegiateFreq(freq, dur), that will create a major scale arpeggio of a given frequency for a given length using your genWave() function. The arguments passed will be as follows:
* freq will be the starting/fundamental frequency in Hz (int or float)
* dur will be the length of time in seconds (int or float)

Reminder: a major scale arpeggio is built from the root, major third, and perfect 5th. You should divide the duration evenly amongst the three notes. 

The function should return a numpy array.

*Hint: your function will use the function(s) you just wrote*

Your function must check for appropriate input and handle any errors

**Bonus: Create a new function called arpegiateNote(scale, dur). Instead of the second input being frequency in Hz, the function should receive a note name like 'C'. You will need to convert this note to frequency. If you want to go the extra mile, make it case sensitive, such that an uppercase 'C' yields a C major arpeggio and lowercase 'c' yields a c minor arpeggio.

In [7]:


def arpeggiateFreq(freq, dur, numHarms=None, A=1.0, phi=0.0, fs=44100.0, type="saw"):

    if not isinstance(freq, (int, float)) or not isinstance(dur, (int, float)):
        raise TypeError("freq and dur must be numeric (int or float)")
    if freq <= 0:
        raise ValueError("freq must be > 0")
    if dur <= 0:
        raise ValueError("dur must be > 0")
    if not isinstance(fs, (int, float)) or fs <= 0:
        raise ValueError("fs must be > 0")

    if numHarms is not None:
        if not isinstance(numHarms, int) or numHarms <= 0:
            raise ValueError("numHarms must be a positive int or None")

    if type not in {"saw", "square", "triangle"}:
        raise ValueError("type must be 'saw', 'square', or 'triangle'")

    ratios = np.array([1.0, 2**(4/12), 2**(7/12)], dtype=float)
    freqs = freq * ratios

    note_dur = dur / 3.0

    n1 = genWave(freqs[0], note_dur, numHarms=numHarms, A=A, phi=phi, fs=fs, type=type)
    n2 = genWave(freqs[1], note_dur, numHarms=numHarms, A=A, phi=phi, fs=fs, type=type)
    n3 = genWave(freqs[2], note_dur, numHarms=numHarms, A=A, phi=phi, fs=fs, type=type)

    return np.concatenate((n1, n2, n3))




## Error handling tips and edge cases
What should happen if the frequency inputs to our functions would generate frequncies above Nyquist?

What should happen if our time inputs are negative? Or are 0?

What should happen if our frequencies are negative? Or are 0?

You should account for actual coding errors and errors that conceptually do not make sense.

### Section 4 - Using functions and graphing (3 points)
Create a 2 second F major arpeggio in an array named 'myArp'. Play back your audio.

In [8]:
fs = 44100
dur = 2

def note_to_midi(note):
    pitch_classes = {
        "C": 0, "C#": 1, "Db": 1,
        "D": 2, "D#": 3, "Eb": 3,
        "E": 4,
        "F": 5, "F#": 6, "Gb": 6,
        "G": 7, "G#": 8, "Ab": 8,
        "A": 9, "A#": 10, "Bb": 10,
        "B": 11
    }
    if note not in pitch_classes:
        raise ValueError("Note must be like C, C#, Db, ... Bb, etc.")
    octave = 4
    return 12 * (octave + 1) + pitch_classes[note]  

def midi_to_freq(midi, A4=440.0):
    return A4 * (2 ** ((midi - 69) / 12))

def pc_to_freq(pc): 
    return midi_to_freq(note_to_midi(pc))

myArp = arpeggiateFreq(_pc_to_freq("F"), dur, fs=fs)
Audio(myArp, rate=fs_))

SyntaxError: unmatched ')' (4002855749.py, line 26)

Remove every 4th sample of 'myArp', save as a new array, 'skipped, and play back your audio. What happened to the audio?

In [ ]:
idx = np.arange(myArp.size)
skipped = myArp[idx % 4 !=3]
Audio(skipped, rate=fs))

#Pitch gets higher and audio quality gets worse

Create a 2 Bb major arpeggio in an array named 'myArp2'. Play back your audio

In [ ]:
myArp2 = arpeggiateFreq(_pc_to_freq("Bb"), dur, fs=fs)
Audio(myArp2, rate=fs)

Reverse 'myArp2' and save as a new array, 'reverse'. Play back your audio

In [ ]:
reverse = myArp2[::-1].copy()
Audio(reverse, rate=fs)

Graph your 4 arrays on top of one another. Make sure to label the axis and plot against time. 

In [ ]:
t_myArp   = np.arange(len(myArp))   / fs
t_skipped = np.arange(len(skipped)) / fs
t_myArp2  = np.arange(len(myArp2))  / fs
t_reverse = np.arange(len(reverse)) / fs

plt.figure()
plt.plot(t_myArp, myArp, label="myArp (F major)")
plt.plot(t_skipped, skipped, label="skipped (remove every 4th)")
plt.plot(t_myArp2, myArp2, label="myArp2 (Bb major)")
plt.plot(t_reverse, reverse, label="reverse (reversed Bb)")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("All four arrays vs time")
plt.legend()
plt.show()

Create another graph, but this time only plot the first "note" of each array

In [ ]:
n1_myArp   = len(myArp)   // 3
n1_skipped = len(skipped) // 3
n1_myArp2  = len(myArp2)  // 3
n1_reverse = len(reverse) // 3

plt.figure()
plt.plot(np.arange(n1_myArp)/fs, myArp[:n1_myArp], label="myArp first note")
plt.plot(np.arange(n1_skipped)/fs, skipped[:n1_skipped], label="skipped first note")
plt.plot(np.arange(n1_myArp2)/fs, myArp2[:n1_myArp2], label="myArp2 first note")
plt.plot(np.arange(n1_reverse)/fs, reverse[:n1_reverse], label="reverse first note")
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("First note only (each array)")
plt.legend()
plt.show()

### GenAI Usage Statement (if applicable)

I used generative AI to assist with understanding python concepts and syntax. The overall work was done individually. 